## Use cases:
1. Generating control sequences for real BAS controllers using CDL
2. FDD and other open loop verification
  * only need input and output points
  * only CDL sequences suffice
  * add annotation tags to CDL blocks (elementary CDL block and composite block) --- higher priority
  * May not need to parse the entire modelica elements. Stop at blocks?
  * User specificies which blocks to parse (user input)
  * Output:
    * semantic model with inputs and output 
    * eventually these will need to be tied to BACnet points. 
  * Questions:
    * how to verify user inputed blocks? 
    * What if there is user defined composite block using a bunch of CDL blocks (from `Buildings.Controls.CDL` or `Buildings.Controls.OBC.*` ) ?
    * who does the BACnet point mapping? 
    * what about the sequences themselves? Are we storing it somehow? 
    * What about documentation? 
    * How to specify that some points will be need to undergo transformation before mapping to CDL input/output
    * Do we allow annotations to propogate to lower blocks? Current CDL has a mechanism to propogate annotations.
3. Developing portable analytics applications
4. Developing portable advanced control sequences
  * Each algorithm would have a BRICK or SPARQL query. 
  * Need Energy model and CDL sequences
  * Output: 
    * semantic model with equipment and points (inputs and outputs)
  * Questions:
    * When should we stop parsing? Currently, I stop as soon as the element type does not start with `Buildings.*`
5. BOPTEST/simulation/Hardware in the loop
  * needs energy model
  * might need external references to BACnet points or other real world variables
  * Output:
    * semantic model with equipment and points (inputs and outputs)

In [1]:
import brickschema
from brickschema.namespaces import BRICK, RDFS, RDF
import json
from element_relationship import Element_Relationship_Extractor


In [2]:
modelica_brick_sensor_type_map = {
    'TemperatureTwoPort': BRICK['Temperature_Sensor'],
    'Temperature': BRICK['Temperature_Sensor'],
    'RelativeTemperature': BRICK['Temperature_Sensor'],
    'TemperatureWetBulbTwoPort': BRICK['Temperature_Sensor'],
    'VolumeFlowRate': BRICK['Flow_Sensor'],
    'RelativeHumidity': BRICK['Humditiy_Sensor'],
    'RelativeHumidityTwoPort': BRICK['Humditiy_Sensor'],
    'Pressure': BRICK['Pressure_Sensor'],
    'RelativePressure': BRICK['Pressure_Sensor']
}

modelica_brick_heat_exchanger_type_map = {
    'DryCoilCounterFlow': BRICK['Heating_Coil'],
    'DryCoilDiscretized': BRICK['Heating_Coil'],
    'DryCoilEffectivenessNTU': BRICK['Heating_Coil'], #DryCoil could also be for cooling in some climate zones, could be part of a heat exchanger,
    'WetCoilCounterFlow': BRICK['Cooling_Coil'],
    'WetCoilDiscretized': BRICK['Cooling_Coil'],
    'EvaporatorCondenser': BRICK['Heat_Exchanger'], #look at which side of compressor it is on to decide if it is evaporator or condensor
    'Heater_T': BRICK['Space_Heater'],
}

modelica_brick_actuator_type_map = {
    'Dampers.Exponential': BRICK['Damper'],
    'Dampers.MixingBox': BRICK['Damper'],
    'Dampers.MixingBoxMinimumFlow': BRICK['Damper'],
    'Dampers.PressureIndependent': BRICK['Damper'],
    'Valves.ThreeWayEqualPercentageLinear': BRICK['Valve'], #need more clarity on the brick side for valves
    'Valves.ThreeWayLinear': BRICK['Valve'],
    'Valves.ThreeWayTable': BRICK['Valve'],
    'Valves.TwoWayEqualPercentageLinear': BRICK['Valve'],
    'Valves.TwoWayLinear': BRICK['Valve'],
    'Valves.TwoWayPolynomial': BRICK['Valve'],
    'Valves.TwoWayPressureIndependent': BRICK['Valve'],
    'Valves.TwoWayQuickOpening': BRICK['Valve'],
    'Valves.TwoWayTable': BRICK['Valve']
}

modelica_brick_mover_type_map = { # anything can be pump or fan according to the media
    'FlowControlled_dp': BRICK['Pump'],
    'FlowControlled_m_flow': BRICK['Pump'],
    'SpeedControlled_Nrpm': BRICK['Fan'],
    'SpeedControlled_y': BRICK['Fan']
}

modelica_brick_thermal_zone_type_map = {
    'Detailed.MixedAir': BRICK['HVAC_Zone'],
    'ReducedOrder.EquivalentAirTemperature': BRICK['HVAC_Zone'],
    'ReducedOrder.RC': BRICK['HVAC_Zone'],
    'ReducedOrer.SolarGain': BRICK['HVAC_Zone']
}

modelica_brick_medium_type_map = {
    'Air': BRICK['Air'],
    'Water': BRICK['Water']
}

In [3]:
with open("config_boptest.json") as fp:
    config = json.load(fp)
config


{'model': 'Buildings.Examples.MultiZoneOfficeSimpleAir.TestCases.TestCase',
 'generate_json': True}

In [4]:
element_extractor = Element_Relationship_Extractor(config_file = "config_boptest.json");

In [5]:
elements, relationships = element_extractor.extract_class_definition()

did not find og_type_specifier=Results type_specifier=Results model=Buildings.Examples.MultiZoneOfficeSimpleAir.BaseClasses.PartialHVAC parent=hvac.
did not find og_type_specifier=BoundaryConditions.WeatherData.Bus type_specifier=BoundaryConditions.WeatherData.Bus model=Buildings.Examples.VAVReheat.BaseClasses.PartialFloor parent=flo.
did not find og_type_specifier=BoundaryConditions.WeatherData.Bus type_specifier=BoundaryConditions.WeatherData.Bus model=Buildings.Examples.VAVReheat.BaseClasses.RoomLeakage parent=flo.leaSou.
did not find og_type_specifier=BoundaryConditions.WeatherData.Bus type_specifier=BoundaryConditions.WeatherData.Bus model=Buildings.Examples.VAVReheat.BaseClasses.RoomLeakage parent=flo.leaEas.
did not find og_type_specifier=BoundaryConditions.WeatherData.Bus type_specifier=BoundaryConditions.WeatherData.Bus model=Buildings.Examples.VAVReheat.BaseClasses.RoomLeakage parent=flo.leaNor.
did not find og_type_specifier=BoundaryConditions.WeatherData.Bus type_specifier=

In [6]:
def get_brick_type(semantic_info):
    if semantic_info != "":
        return BRICK[semantic_info.split(" ")[-1].split(":")[1]]
    return ""

In [9]:
brick_graph = brickschema.Graph(load_brick=True)
all_points = list(brick_graph.transitive_subjects(object=BRICK['Point'], predicate=RDFS['subClassOf']))
all_zones = list(brick_graph.transitive_subjects(object=BRICK['Zone'], predicate=RDFS['subClassOf']))
all_equipment = list(brick_graph.transitive_subjects(object=BRICK['Equipment'], predicate=RDFS['subClassOf']))
all_systems = list(brick_graph.transitive_subjects(object=BRICK['System'], predicate=RDFS['subClassOf']))

for item in elements:
    type_specifier = elements[item]['type_specifier']
    semantic = elements[item].get('semantic', '')
    parent_semantic = ""
    if '.' in item:
        parent  = item.rsplit(".", 1)[0]
        parent_semantic = get_brick_type(elements[parent].get("semantic"))

    if semantic != "":
        brick_type = get_brick_type(semantic)
        print(item, 'a', brick_type)
        if brick_type in all_points:
            if parent_semantic in all_equipment or parent_semantic in all_zones or parent_semantic in all_systems:
                print(parent, BRICK['hasPoint'], item)
        if brick_type in all_zones:
            if parent_semantic in all_zones:
                print(parent, BRICK['hasPart'], item)
        if brick_type in all_equipment:
            if parent_semantic in all_equipment or parent_semantic in all_systems:
                print(parent, BRICK['hasPart'], item)
            if parent_semantic in all_zones:
                print(parent, BRICK['hasLocation'], item)
            
        if item in relationships:
            from_item_og = item
            from_item = item
        else:
            for fro in relationships:
                if "." in fro and fro.rsplit(".", 1)[0] == item:
                    from_item_og = fro
                    from_item = fro.rsplit(".", 1)[0]
                    break
        
        for to_item_og in relationships[from_item_og]:
            to_semantic = ""
            to_item = None
            if to_item_og in elements:
                to_item = to_item_og
            elif to_item_og.rsplit(".", 1)[0] in elements:
                to_item = to_item_og.rsplit(".", 1)[0]
            to_semantic = elements[to_item].get('semantic')
            
            if to_semantic != "":
                to_brick_type = get_brick_type(to_semantic)
                if brick_type in all_equipment:
                    if to_brick_type in all_equipment:
                        if from_item_og != from_item and to_item_og != to_item:
                            from_port = from_item_og.rsplit(".", 1)[1]
                            to_port = to_item_og.rsplit(".", 1)[1]
                            
                            if (from_port.startswith("port_b") or from_port.startswith("port2") or from_port.startswith("portb") or from_port.startswith("port_2")) and \
                                (to_port.startswith("port_a") or to_port.startswith("port1") or to_port.startswith("porta") or to_port.startswith("port_1")):
                                print(from_item, BRICK['feeds'], to_item)
                    if to_brick_type in all_points:
                        print(from_item, BRICK['hasPoint'], to_item)

chi a https://brickschema.org/schema/Brick#Chiller
hvac.heaCoi a https://brickschema.org/schema/Brick#Heating_Coil
hvac https://brickschema.org/schema/Brick#hasPart hvac.heaCoi
hvac.heaCoi https://brickschema.org/schema/Brick#feeds hvac.cooCoi
hvac.cooCoi a https://brickschema.org/schema/Brick#Cooling_Coil
hvac https://brickschema.org/schema/Brick#hasPart hvac.cooCoi
hvac.cooCoi https://brickschema.org/schema/Brick#feeds hvac.fanSup
hvac.fanSup a https://brickschema.org/schema/Brick#Supply_Fan
hvac https://brickschema.org/schema/Brick#hasPart hvac.fanSup
hvac.fanSup https://brickschema.org/schema/Brick#hasPoint hvac.dpDisSupFan
hvac.dpDisSupFan a https://brickschema.org/schema/Brick#Discharge_Air_Static_Pressure_Sensor
hvac https://brickschema.org/schema/Brick#hasPoint hvac.dpDisSupFan
hvac.VOut1 a https://brickschema.org/schema/Brick#Outside_Air_Flow_Sensor
hvac https://brickschema.org/schema/Brick#hasPoint hvac.VOut1
hvac.cor a https://brickschema.org/schema/Brick#VAV
hvac https://br

In [45]:
# brick_types = {}
# for element in a:
#     type_specifier = a[element].get('type_specifier')
#     brick_type = None
    
#     if type_specifier.startswith("Buildings.Fluid.Sensors"):
#         brick_type = modelica_brick_sensor_type_map.get(type_specifier.split('Buildings.Fluid.Sensors.')[1], None)

#     if type_specifier.startswith("Buildings.Fluid.HeatExchangers"):
#         brick_type = modelica_brick_heat_exchanger_type_map.get(type_specifier.split('Buildings.Fluid.HeatExchangers.')[1], None)
    
#     if type_specifier.startswith("Buildings.Fluid.Actuators"):
#         brick_type = modelica_brick_actuator_type_map.get(type_specifier.split('Buildings.Fluid.Actuators.')[1], None)
        
#     if type_specifier.startswith("Buildings.Fluid.Movers"):
#         brick_type = modelica_brick_mover_type_map.get(type_specifier.split('Buildings.Fluid.Movers.')[1], None)
        
#     if type_specifier.startswith("Buildings.ThermalZones"):
#         brick_type = modelica_brick_thermal_zone_type_map.get(type_specifier.split('Buildings.ThermalZones.')[1], None)
    
#     if type_specifier.startswith("Buildings.Media"):
#         brick_type = modelica_brick_medium_type_map.get(type_specifier.split('Buildings.Media.')[1], None)
    
#     if brick_type is not None:
#         brick_types[element] = brick_type